In [1]:
!pip install pyspark

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import split, concat_ws

In [3]:
conf = SparkConf().setAppName("191805056_EsmanurDeli")
sc = SparkContext.getOrCreate(conf=conf)

23/05/22 16:37:14 WARN Utils: Your hostname, Esmanur-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.159.223 instead (on interface en0)
23/05/22 16:37:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 16:37:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dataset = sc.textFile("dataset15k.csv")

In [5]:
header = dataset.first()

In [6]:
# İlk satırı hariç diğer satırları seç
data = dataset.filter(lambda line: line != header)

In [7]:
# Kolon adıyla birleştir
dataset = sc.parallelize([header]).union(data)

In [8]:
num_books = dataset.count()
print("Number of Books: ", num_books)

Number of Books:  34215


In [9]:
book_fields = dataset.map(lambda lines: lines.split(","))
isbn = book_fields.map(lambda fields:fields[4])

In [10]:
isbn_filtered = isbn.map(lambda x: int(x) if x.isdigit() else None).filter(lambda x: x is not None)


In [11]:
book_fields.take(2)

[['Unnamed: 0',
  'user_id',
  'location',
  'age',
  'isbn',
  'rating',
  'book_title',
  'book_author',
  'year_of_publication',
  'publisher',
  'img_s',
  'img_m',
  'img_l',
  'Summary',
  'Language',
  'Category',
  'city',
  'state',
  'country'],
 ['0',
  '2',
  '"stockton',
  ' california',
  ' usa"',
  '18.0',
  '0195153448',
  '0',
  'Classical Mythology',
  'Mark P. O. Morford',
  '2002.0',
  'Oxford University Press',
  'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg',
  'http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg',
  'http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg',
  '"Provides an introduction to classical myths placing the addressed']]

In [12]:
isbn.take(5)

23/05/22 16:37:37 ERROR Executor: Exception in task 4.0 in stage 7.0 (TID 30)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/rdd.py", line 2830, in takeUpToNumLeft
    yield next(iterator)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 7.0 failed 1 times, most recent failure: Lost task 3.0 in stage 7.0 (TID 29) (172.16.159.223 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Library/Frameworks/Python